In [1]:
import cftime
import glob
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
from scipy import stats

import mission_tools

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [6]:
mission_list = glob.glob('../microSWIFT_data/final_dataset/mission_*.nc')

xFRF_break_locs = []
yFRF_break_locs = []
xFRF_break_locs_norm = []
break_times_all = []
mission_number = []
microSWIFT_nums = []

gamma = 0.35
awac_file = '../microSWIFT_data/FRFdata/FRF-ocean_waves_8m-array_202110.nc'
bathy_file = '../microSWIFT_data/FRFdata/FRF_geomorphology_DEMs_surveyDEM_20211021.nc'
bathy_dataset = nc.Dataset(bathy_file)
xFRF_grid, yFRF_grid = np.meshgrid(bathy_dataset['xFRF'][:],bathy_dataset['yFRF'][:])
bathy = bathy_dataset['elevation'][0,:,:]
waterlevel_file = '../microSWIFT_data/FRFdata/FRF-ocean_waterlevel_eopNoaaTide_202110.nc'
waterlevel_dataset = nc.Dataset(waterlevel_file)
waterlevel = waterlevel_dataset['waterLevel'][:]
waterlevel_time = waterlevel_dataset['time'][:]


for mission_nc in mission_list:
    mission_dataset = nc.Dataset(mission_nc, mode='r')

    for microswift_num in range(mission_dataset['acceleration_z_body'].shape[0]):
        a_z_test = mission_dataset['acceleration_z_body'][microswift_num,:]
        xFRF_test = mission_dataset['xFRF'][microswift_num,:]
        yFRF_test = mission_dataset['yFRF'][microswift_num,:]
        mission_time = np.median(mission_dataset['time'][:])
        time = mission_dataset['time'][:]
        a_z_test_demean = a_z_test - np.nanmean(a_z_test)
        time_seconds = np.arange(a_z_test.size)/12

        # reshape time series into 5 second windows and compute variance
        window_size = 60 # 5 seconds at 12 Hz
        a_z_test_reshape = np.reshape(a_z_test_demean[:(a_z_test_demean.size//window_size * window_size)],
                                    (a_z_test_demean.size//window_size, window_size))
        time_seconds_reshape = np.reshape(time_seconds[:(time_seconds.size//window_size * window_size)],
                                        (time_seconds.size//window_size, window_size))
        xFRF_reshape = np.reshape(xFRF_test[:(xFRF_test.size//window_size * window_size)],
                                        (xFRF_test.size//window_size, window_size))
        yFRF_reshape = np.reshape(yFRF_test[:(yFRF_test.size//window_size * window_size)],
                                        (yFRF_test.size//window_size, window_size))
        time_reshape = np.reshape(time[:(time.size//window_size * window_size)],
                                        (time.size//window_size, window_size))
        a_z_test_window_var = np.nanvar(a_z_test_reshape, axis=1)
        time_seconds_window = np.nanmean(time_seconds_reshape, axis=1)
        time_window = np.nanmean(time_reshape, axis=1)
        xFRF_window = np.nanmean(xFRF_reshape, axis=1)
        yFRF_window = np.nanmean(yFRF_reshape, axis=1)
        total_varaince = np.nanvar(a_z_test_demean)

        # Find Indices where the variance spikes 
        acceleration_peak_threshold = 2.5
        above_thresh_inds = np.where(a_z_test_window_var > acceleration_peak_threshold*total_varaince)
        
        # add x and y break locations
        xFRF_break_locs.append(xFRF_window[above_thresh_inds].flatten())
        yFRF_break_locs.append(yFRF_window[above_thresh_inds].flatten())

        # add the time value 
        break_times_all.append(time_window[above_thresh_inds].flatten())

        # Compute depth that waves should be breaking in 
        mission_hs = mission_tools.closest_awac_sig_wave_height(mission_time, awac_file)
        mission_break_depth = mission_hs / gamma
        mission_xs_gamma_location = np.interp(-mission_break_depth, np.flip(np.nanmean(bathy, axis=0)), np.flip(bathy_dataset['xFRF'][:]))

        # Normalize the cross shore positions by the edge of surfzone estimate
        xFRF_break_locs_norm.append(xFRF_window[above_thresh_inds].flatten()/mission_xs_gamma_location)
        
        # Save mission number
        mission_number.append(int(mission_nc[41:-3]) * np.ones(time_window[above_thresh_inds].flatten().size))

        # Save the trajectory number 
        microSWIFT_nums.append(microswift_num * np.ones(time_window[above_thresh_inds].flatten().size))
        
xFRF_break_locs = np.ma.concatenate(xFRF_break_locs).flatten()
xFRF_break_locs_norm = np.ma.concatenate(xFRF_break_locs_norm).flatten()
yFRF_break_locs = np.ma.concatenate(yFRF_break_locs).flatten()
break_times_all = np.ma.concatenate(break_times_all).flatten()
mission_number = np.ma.concatenate(mission_number).flatten()
microSWIFT_nums = np.ma.concatenate(microSWIFT_nums).flatten()

/opt/homebrew/Caskroom/mambaforge/base/envs/dunex-venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:784: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/var/folders/v_/wf8btpmn6s588vb9xbxbr9bw0000gp/T/ipykernel_50729/3002039828.py:46: RuntimeWarning: Degrees of freedom <= 0 for slice.
  a_z_test_window_var = np.nanvar(a_z_test_reshape, axis=1)
/var/folders/v_/wf8btpmn6s588vb9xbxbr9bw0000gp/T/ipykernel_50729/3002039828.py:49: RuntimeWarning: Mean of empty slice
  xFRF_window = np.nanmean(xFRF_reshape, axis=1)
/var/folders/v_/wf8btpmn6s588vb9xbxbr9bw0000gp/T/ipykernel_50729/3002039828.py:50: RuntimeWarning: Mean of empty slice
  yFRF_window = np.nanmean(yFRF_reshape, axis=1)


In [8]:
# Create a pandas dataframe from all wave data
breaker_df = pd.DataFrame(cftime.num2pydate(break_times_all, units=mission_dataset['time'].units,
                       calendar=mission_dataset['time'].calendar), columns=['time'])
breaker_df['mission number'] = mission_number
breaker_df['trajectory number'] = microSWIFT_nums
breaker_df['cross shore location [m]'] = xFRF_break_locs
breaker_df['cross shore location normalized [m]'] = xFRF_break_locs_norm
breaker_df['along shore location [m]'] = yFRF_break_locs


breaker_df


,time,mission number,trajectory number,cross shore location [m],cross shore location normalized [m],along shore location [m]
0,2021-10-29 14:32:02.457564,78.0,0.0,342.912569,0.644558,650.797664
1,2021-10-29 14:32:32.457444,78.0,0.0,309.671613,0.582076,671.188170
2,2021-10-29 14:34:57.456864,78.0,0.0,287.717313,0.540810,778.737823
3,2021-10-29 14:35:27.456744,78.0,0.0,223.654297,0.420393,799.877935
4,2021-10-29 14:35:47.456664,78.0,0.0,208.835801,0.392539,814.075185
...,...,...,...,...,...,...
3414,2021-10-05 18:21:42.456243,4.0,7.0,188.567424,1.030392,80.595533
3415,2021-10-05 18:13:12.458284,4.0,8.0,199.447297,1.089843,-61.277648
3416,2021-10-14 13:00:27.458224,34.0,0.0,199.005140,0.855444,386.044583
3417,2021-10-14 13:14:42.454804,34.0,0.0,205.019366,0.881297,480.649971


In [9]:
breaker_df.to_csv('../microSWIFT_data/breaker_df.csv')